# Предсказание спроса на такси# 

Контакты:

 *e-mail*: anatoliy.chudakov020@gmail.com

 *slack*: 

## 1.Описание задачи и датасета##

**Актуальность**. Когда пользователь вызвал такси, он ожидает получить машину как можно быстрее. Оператор такси может удовлетворять
такие требования максимально эффективно, если имеет прогнозные данные о спросе на такси в районе пользователя. Данные о прогнозе спроса на такси позволяют принимать более оптимальные решения о назначении водителя путём нахождения компромиссного решения между требованиями пользователей, комиссией агрегатора и вознаграждением водителя, зависящим от загруженности в районе.

Если рассматривать актуальность шире, то решение этой задачи математически схоже с задачами прогнозирования спроса на товары для ритейла, цены акций для финансовых институтов, онлайн-активности аудитории для IT и телекома и так далее. Во всех этих случаях мы будем иметь дело с прогнозированием целевого признака в многомерных(или одномерных) временных рядах. Безусловно, каждая индустрия имеет свою специфику и конкретные задачи могут слегка различаться. Однако, очевидно то, что общие принципы решения подобных задач, которые будут продемонстрированы в этой работе, будут схожи для любого типа компании, которой необходимо получить прогноз одного или нескольких целевых признаков в зависимости от значения этого признака в прошлом.   

**Целевая переменная**: y - количество вызовов водителей такси.

**Признаки**: id - id наблюдения.
 
Исходный датасет можно найти [здесь](https://www.kaggle.com/c/yellowtaxi/data).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
series = pd.read_csv('series.csv', encoding='utf-8')

In [3]:
series.head()

,id,y
0,1075_2016-04-30_18_6,71
1,1075_2016-04-30_19_5,71
2,1075_2016-04-30_19_6,37
3,1075_2016-04-30_20_4,71
4,1075_2016-04-30_20_5,37


## 2. Создание первой группы признаков

Выделю признаки из столбца id:
    - локация
    - дата в формате год-месяц-день
    - день недели
    - номер часа
    - номер минуты
    - выходной(булево значение)
Из признака "локация" получу дамми-переменные.

In [4]:
import re
import datetime

In [5]:
series['location'] = series['id'].str.extract(r'(^\d*)')

In [6]:
series['date'] = series['id'].str.extract(r'(\d{4}-\d{1,2}-\d{1,2})')

In [7]:
series['date'] = pd.to_datetime(series['date'])

In [8]:
series['date'].value_counts()

2016-05-05    14688
2016-05-24    14688
2016-05-09    14688
2016-05-16    14688
2016-05-23    14688
2016-05-30    14688
2016-05-08    14688
2016-05-22    14688
2016-05-03    14688
2016-05-17    14688
2016-05-21    14688
2016-05-28    14688
2016-05-11    14688
2016-05-18    14688
2016-05-20    14688
2016-05-01    14688
2016-05-29    14688
2016-05-10    14688
2016-05-19    14688
2016-05-06    14688
2016-05-13    14688
2016-05-27    14688
2016-05-12    14688
2016-05-26    14688
2016-05-07    14688
2016-05-14    14688
2016-05-04    14688
2016-05-25    14688
2016-05-15    14688
2016-05-02    14688
2016-05-31    12546
2016-04-30     2142
Name: date, dtype: int64

Наблюдения за 2016-04-30 и 2016-05-31 представлены с интервалом, отличным от других дат. Удалю эти наблюдения.

In [9]:
series.drop(series[(series.date == '2016-04-30') | (series.date == '2016-05-31')].index, inplace=True)

In [10]:
series['week_day'] = series['date'].dt.weekday

In [11]:
series['is_weekend'] = np.where((series['week_day'] == 5) | (series['week_day'] ==6) | 
                               (series['date'].dt.day == 2) | (series['date'].dt.day == 3) |
                                (series['date'].dt.day == 9),
                               1, 0)

In [12]:
series['hour_num'] = series['id'].str.extract(r'\d*_\d*-\d*-\d*_(\d{1})')

In [13]:
series['min_num'] = series['id'].str.extract(r'(\d$)')

In [14]:
series = pd.get_dummies(series, columns=['location'])

## 3. Предобработка датасета

Переименую столбец с количеством заказов и сменю тип на float32.

In [15]:
series = series.rename(columns={'y': 'orders'})

In [16]:
pd.to_numeric(series['orders'], downcast='float')

21        37.0
22        10.0
23        14.0
24         6.0
25         8.0
          ... 
455200    75.0
455201     0.0
455202     0.0
455203     3.0
455204    89.0
Name: orders, Length: 440640, dtype: float32

Заполню значения переменных, обозначающих район, количеством соответствующих заказов.

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [370]:
pd.set_option('display.max_columns', None) 

In [37]:
def get_location_values(data, sub='', column=''):
    for key, value in data.iteritems():
            
        if key.startswith(sub):
            data[key] = np.where(data[key] == 1, data[column], np.nan)

In [48]:
get_location_values(data=series, sub='location_', column='orders')

In [42]:
def shift_values(data, sub='', location='', column=''):
    count = 0
    position = data.shape[0] - data[location].isna().sum()
        
    for key, value in data.iteritems():
            
        if key.startswith(sub):
            count += 1
                
            if count == 2:
                data.loc[:, key] = data.loc[:, key].shift(-position)
            elif count > 2:
                position += data.shape[0] - data[location].isna().sum()
                data.loc[:, key] = data.loc[:, key].shift(-position)
            elif count == 1:
                continue

In [49]:
shift_values(data=series, sub='location_', location='location_1075', column='orders')

Удалю ненужные строки с NaN значениями.

In [50]:
series.dropna(inplace=True)

In [51]:
series.shape

(4320, 109)

И запишу результаты в файл.

In [52]:
series.to_csv('series_prepared.csv', encoding='utf-8', index=False)

## 4. Анализ данных

### 4.1. Стационарность